# Prepare and understand MLS stratospheric water vapour

We will use this as estimate for Hunga Tonga positive forcing.

To obtain the data, go to https://search.earthdata.nasa.gov/search?q=MLS%20H2O&fi=MLS&fl=3%2B-%2BGridded%2BObservations. Select the MLS/Aura Level 3 Monthly Binned Water Vapor (H2O) Mixing Ratio on Assorted Grids V005 (ML3MBH2O). Download this data to ../data/mls.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as pl
import numpy as np
import glob
from fair.earth_params import mass_atmosphere

In [ ]:
data = np.ones((19*12, 37)) * np.nan

In [ ]:
for year in range(2004, 2023):
    nc = Dataset(glob.glob(f'../data/mls/MLS-Aura_L3MB-H2O_*_{year}.nc')[0])
    data[(year-2004)*12:(year-2004)*12+12, :] = nc.groups['H2O PressureZM'].variables['value'][:, 12:49, 15:30].mean(axis=2)
    plev = nc.groups['H2O PressureZM'].variables['lev'][12:49]
    nc.close()

In [ ]:
data

In [ ]:
X, Y = np.meshgrid(np.arange(2004+1/24, 2023, 1/12), plev)

In [ ]:
data[data==0] = np.nan

In [ ]:
pl.contourf(
    np.arange(2004+1/24, 2023, 1/12),
    plev[:],
    data.T
)
ax = pl.gca()
ax.set_ylim(ax.get_ylim()[::-1])
pl.colorbar()

In [ ]:
plev

In [ ]:
data[179]  # Dec 2018 NaN

## Need to conservatively regrid this to ERA5's plev / lon / lat

I regridded ERA5 to 2 degrees to run in SOCRATES, then we want H2O in terms of specific humidity (kg / kg) for 10, 20, 30 hPa with bounds (7.5, 15), (15, 25), (25, 40).

In [ ]:
data = np.ma.masked_all((19*12, 37, 72, 45)) * np.nan
data

In [ ]:
for year in range(2004, 2023):
    nc = Dataset(glob.glob(f'../data/mls/MLS-Aura_L3MB-H2O_*_{year}.nc')[0])
    data[(year-2004)*12:(year-2004)*12+12, ...] = nc.groups['H2O PressureGrid'].variables['value'][:, 12:49, ...]
    plev = nc.groups['H2O PressureGrid'].variables['lev'][12:49]
    lat = nc.groups['H2O PressureGrid'].variables['lat'][:]
    lon = nc.groups['H2O PressureGrid'].variables['lon'][:]
    lat_bnds = nc.groups['H2O PressureGrid'].variables['lat_bnds'][:]
    lon_bnds = nc.groups['H2O PressureGrid'].variables['lon_bnds'][:]
    nc.close()

In [ ]:
mass_atmosphere # slice

In [ ]:
p_top = 7.5
p_bottom = 40
mass_slice = (p_bottom-p_top)/1000 * mass_atmosphere

In [ ]:
mass_slice  # how much does the stratospheric slice weigh in kg

In [ ]:
plev[1:19]

In [ ]:
plev_bounds = 0.5 * (plev[:-1] + plev[1:])
plev_bounds

In [ ]:
# from scipy.interpolate import interp1d

In [ ]:
# levnum = np.arange(16, 28)
# intfun = interp1d(plev, levnum)
# intfun([40, 25, 15, 7.5])

In [ ]:
# the 30 hPa level is 

In [ ]:
data[178, :, 0, 22]

In [ ]:
plev

In [ ]:
plev_bounds = 0.5 * (plev[1:] + plev[:-1])
plev_bounds

In [ ]:
plev_diff = -np.diff(plev)
plev_thick = 0.5 * (plev_diff[1:] + plev_diff[:-1])
plev_thick

In [ ]:
lon_lat_weight = np.diff(np.sin(np.radians(lat_bnds))).squeeze() / 2 / 72

In [ ]:
# mass of water vapour at each cell
mass_h2o = (plev_thick[None, :, None, None] / 1000 * mass_atmosphere * lon_lat_weight * data[:, 1:-1, ...]) * 18.015 / 28.97
# the last ratio is VMR to MMR
mass_h2o.shape

In [ ]:
mass_h2o.sum(axis=(1,2,3))  # in kg
mass_h2o.sum(axis=(1,2,3))/1e9  # in Tg

In [ ]:
pl.plot(mass_h2o.sum(axis=(1,2,3))/1e9)

In [ ]:
# climatology
mass_h2o_20042021 = np.ma.masked_all((12))
for month in range(12):
    mass_h2o_20042021[month] = np.nanmean(mass_h2o[month:216:12].sum(axis=(1,2,3))/1e9)

In [ ]:
mass_h2o_2022 = np.ma.masked_all(12)
for month in range(12):
    mass_h2o_2022[month] = mass_h2o[month+216].sum()/1e9

In [ ]:
mass_h2o_2022

In [ ]:
(mass_h2o_2022 - mass_h2o_20042021)

In [ ]:
plev_diff

In [ ]:
data[-1, :, 0, 23]

In [ ]:
# Then I need to convert this back to a mass mixing ratio for socrates

In [ ]:
nc = Dataset(glob.glob(f'../data/mls/MLS-Aura_L3MB-H2O_*_{year}.nc')[0])

In [ ]:
nc.groups['H2O PressureGrid'].variables

In [ ]:
nc.close()